# Publications markdown generator for academicpages

Takes a TSV of publications with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `publications.py`. Run either from the `markdown_generator` folder after replacing `publications.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases of citations, rather than Stuart's non-standard TSV format and citation style.


## Data format

The TSV needs to have the following columns: pub_date, title, venue, excerpt, citation, site_url, and paper_url, with a header at the top. 

- `excerpt` and `paper_url` can be blank, but the others must have values. 
- `pub_date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/publications/YYYY-MM-DD-[url_slug]`

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [2]:
!cat publications.tsv

pub_date	title	venue	excerpt	citation	site_url	paper_url
2023-08-22	Convolutional Neural Network-Based Identifying Gender of Kiwifruit Flowers in Autonomous Pollination for Future Farming	International Conference on Mechatronics and Intelligent Robotics		"Tian, Yi, Huang, Ye, Chen, Yi. ""Convolutional Neural Network-Based Identifying Gender of Kiwifruit Flowers in Autonomous Pollination for Future Farming."" International Conference on Mechatronics and Intelligent Robotics, 2023."	https://link.springer.com/10.1007/978-981-99-8498-5	
2024-01-01	BiF³-Net: A Full BiFormer Full-scale Fusion Network for Accurate Gastrointestinal Images Segmentation	Medical Imaging with Deep Learning		"Wang, Yunze, Chen, Silin, Tian, Yi, Wang, Tianyang, Liu, Jingxin. ""BiF³-Net: A Full BiFormer Full-scale Fusion Network for Accurate Gastrointestinal Images Segmentation."" Medical Imaging with Deep Learning, 2024."	https://openreview.net/pdf?id=kIRMpUufhm	
2025-04-14	Advancing cross-organ domain generalizatio

## Import pandas

We are using the very handy pandas library for dataframes.

In [3]:
import pandas as pd

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [7]:
publications = pd.read_csv("final_publications.tsv", sep="\t", header=0)
publications


,pub_date,title,venue,excerpt,citation,site_url,paper_url,url_slug,slides_url
0,2023-08-22,Convolutional Neural Network-Based Identifying...,International Conference on Mechatronics and I...,NaN,"Tian, Yi, Huang, Ye, Chen, Yi. ""Convolutional ...",https://link.springer.com/10.1007/978-981-99-8...,NaN,convolutional-neural-network-based-identifying,NaN
1,2024-01-01,BiF³-Net: A Full BiFormer Full-scale Fusion Ne...,Medical Imaging with Deep Learning,NaN,"Wang, Yunze, Chen, Silin, Tian, Yi, Wang, Tian...",https://openreview.net/pdf?id=kIRMpUufhm,NaN,bif-net-a-full-biformer,NaN
2,2025-04-14,Advancing cross-organ domain generalization wi...,2025 IEEE 22nd International Symposium on Biom...,NaN,"Meng, Biwen, Long, Xi, Yang, Wanrong, Liu, Ruo...",http://arxiv.org/abs/2503.18567,NaN,advancing-cross-organ-domain-generalization,NaN
3,2025-12-25,CellMamba: Adaptive Mamba for Accurate and Eff...,The British Machine Vision Conference (BMVC) 2025,NaN,"Yi Tian, Ruochen Liu, et al. ""CellMamba: Adapt...",https://bmva-archive.org.uk/bmvc/2025/assets/p...,https://bmva-archive.org.uk/bmvc/2025/assets/p...,cellmamba-adaptive-mamba-for-accurate,NaN


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [8]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    """Produce entities within text."""
    return "".join(html_escape_table.get(c,c) for c in text)

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [9]:
import os
for row, item in publications.iterrows():
    
    md_filename = str(item.pub_date) + "-" + item.url_slug + ".md"
    html_filename = str(item.pub_date) + "-" + item.url_slug
    year = item.pub_date[:4]
    
    ## YAML variables
    
    md = "---\ntitle: \""   + item.title + '"\n'
    
    md += """collection: publications"""
    
    md += """\npermalink: /publication/""" + html_filename
    
    if len(str(item.excerpt)) > 5:
        md += "\nexcerpt: '" + html_escape(item.excerpt) + "'"
    
    md += "\ndate: " + str(item.pub_date) 
    
    md += "\nvenue: '" + html_escape(item.venue) + "'"
    
    if len(str(item.slides_url)) > 5:
        md += "\nslidesurl: '" + item.slides_url + "'"

    if len(str(item.paper_url)) > 5:
        md += "\npaperurl: '" + item.paper_url + "'"
    
    md += "\ncitation: '" + html_escape(item.citation) + "'"
    
    md += "\n---"
    
    ## Markdown description for individual page
        
    if len(str(item.excerpt)) > 5:
        md += "\n" + html_escape(item.excerpt) + "\n"

    if len(str(item.slides_url)) > 5:
        md += "\n[Download slides here](" + item.slides_url + ")\n" 

    if len(str(item.paper_url)) > 5:
        md += "\n[Download paper here](" + item.paper_url + ")\n" 
        
    md += "\nRecommended citation: " + item.citation
    
    md_filename = os.path.basename(md_filename)
       
    with open("../_publications/" + md_filename, 'w') as f:
        f.write(md)

These files are in the publications directory, one directory below where we're working from.

In [10]:
!ls ../_publications/

2009-10-01-paper-title-number-1.md
2010-10-01-paper-title-number-2.md
2015-10-01-paper-title-number-3.md
2023-08-22-convolutional-neural-network-based-identifying.md
2024-01-01-bif-net-a-full-biformer.md
2024-02-17-paper-title-number-4.md
2025-04-14-advancing-cross-organ-domain-generalization.md
2025-06-08-paper-title-number-5.md
2025-12-25-cellmamba-adaptive-mamba-for-accurate.md


In [11]:
!cat ../_publications/2025-12-25-cellmamba-adaptive-mamba-for-accurate.md

---
title: "CellMamba: Adaptive Mamba for Accurate and Efficient Cell Detection"
collection: publications
permalink: /publication/2025-12-25-cellmamba-adaptive-mamba-for-accurate
date: 2025-12-25
venue: 'The British Machine Vision Conference (BMVC) 2025'
paperurl: 'https://bmva-archive.org.uk/bmvc/2025/assets/papers/Paper_173/paper.pdf'
citation: 'Yi Tian, Ruochen Liu, et al. &quot;CellMamba: Adaptive Mamba for Accurate and Efficient Cell Detection.&quot; BMVC 2025.'
---
[Download paper here](https://bmva-archive.org.uk/bmvc/2025/assets/papers/Paper_173/paper.pdf)

Recommended citation: Yi Tian, Ruochen Liu, et al. "CellMamba: Adaptive Mamba for Accurate and Efficient Cell Detection." BMVC 2025.